In [1]:
import re

import pandas as pd
from vietadminunits import parse_address, get_data
from vietadminunits.utils import to_key, to_alphanumeric
import pickle

In [2]:
df = pd.DataFrame(get_data())

In [3]:
df['long_province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['long_district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['long_ward_alphanumeric'] = df['long_ward'].fillna('').apply(to_alphanumeric)

df['province_alphanumeric'] = df['province'].apply(to_alphanumeric)
df['short_district_alphanumeric'] = df['short_district'].apply(to_alphanumeric)
df['short_ward_alphanumeric'] = df['short_ward'].fillna('').apply(to_alphanumeric)

In [4]:
for ward in ['long_ward_alphanumeric', 'short_ward_alphanumeric']:
    for district in ['long_district_alphanumeric', 'short_district_alphanumeric']:
        for province in ['long_province_alphanumeric', 'province_alphanumeric']:
            df[f"address_{ward}_{district}_{province}"] = df[ward] + df[district] + df[province]
            df[f"address_comma_{ward}_{district}_{province}"] = df[ward] + ',' + df[district] + ',' + df[province]

In [5]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['district'].apply(to_key, args=(2,))
df['ward_key'] = df['ward'].apply(to_key, args=(3,))

In [6]:
address_cols = [col for col in df.columns if 'address' in col]

## Find wards that are the same keyword with provinces

In [ ]:
wrong_provinces = []



for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)
        unit = parse_address(address, level=1)
        data = {
            'address': address,
            'province': row.province,
            'province_key': row.province_key,
            'wrong_province_key': unit.province_key,
        }
        
        if unit.province == None:
            print(address)
            wrong_provinces.append(data)
            
        elif unit.province_key != row.province_key:
            print(address)
            wrong_provinces.append(data)

In [ ]:
df_wrong_provinces = pd.DataFrame(wrong_provinces)

In [ ]:
df_wrong_provinces.info()

In [ ]:
df_wrong_provinces

In [ ]:
# df_wrong_provinces[df_wrong_provinces.wrong_province_key.isna()]

In [ ]:
DICT_double_check_provinces = {}
for row in df_wrong_provinces.itertuples():
    DICT_double_check_provinces[row.wrong_province_key] = df_wrong_provinces[df_wrong_provinces['wrong_province_key']==row.wrong_province_key]['province_key'].unique().tolist()

In [ ]:
DICT_double_check_provinces

## Find wards that are the same keyword with districts

We have to add `double_check_provinces` to module to solve province level before doing this step.

In [ ]:
wrong_districts = []
for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)

        unit = parse_address(address, level=3)
    
        data = {
            'address': address,
            'province_english': row.province_english,
            'province_key': row.province_key,
            'wrong_province_key': unit.province_key,
            'district_key': row.district_key,
            'wrong_district_key': unit.district_key,
        }
        
        if unit.province == None:
            print(address)
            wrong_districts.append(data)
        elif unit.district == None:
            print(address)
            wrong_districts.append(data)
                
        elif unit.district_key != row.district_key:
            print(address)
            wrong_districts.append(data)



In [ ]:
df_wrong_districts = pd.DataFrame(wrong_districts)

In [ ]:
df_wrong_districts = df_wrong_districts[df_wrong_districts.province_key==df_wrong_districts.wrong_province_key]

In [ ]:
df_wrong_districts

In [ ]:
df_wrong_districts.info()

In [ ]:
df_wrong_districts[df_wrong_districts.wrong_district_key.isna()]

In [ ]:
DICT_double_check_districts = {}

for province_english in df_wrong_districts['province_english'].unique():
    district_data = {}
    for wrong_district_key in df_wrong_districts[(df_wrong_districts['province_english']==province_english)]['wrong_district_key'].unique():
        tmp_district_keys = df_wrong_districts[(df_wrong_districts['province_english']==province_english) & (df_wrong_districts['wrong_district_key']==wrong_district_key)]['district_key'].unique().tolist()
        
        tmp_district_keys = sorted(tmp_district_keys, key=len, reverse=True)
        district_data[wrong_district_key] = tmp_district_keys
    
    district_data = dict(sorted(district_data.items(), key=lambda x: len(x[0]), reverse=True))
    DICT_double_check_districts[province_english] = district_data

In [ ]:
DICT_double_check_districts

## Double check ward

In [7]:
wrong_wards = []
for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)
        
        try:
        
            unit = parse_address(address, level=3)
        
            data = {
                'address': address,
                'province_english': row.province_english,
                'province_key': row.province_key,
                'wrong_province_key': unit.province_key,
                'district_english': row.district_english,
                'district_key': row.district_key,
                'wrong_district_key': unit.district_key,
                'ward_key': row.ward_key,
                'wrong_ward_key': unit.ward_key,
            }
            
            if unit.province == None:
                print(address)
                wrong_wards.append(data)
                
            elif unit.district == None:
                print(address)
                wrong_wards.append(data)
                
            elif (unit.ward == None) and (unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']):
                print(address)
                wrong_wards.append(data)
                    
            elif (unit.ward_key != row.ward_key) and (unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']):
                print(address)
                wrong_wards.append(data)
                
                
        except Exception as e:
            print('ERROR address:', address)
            print('ERROR info:', e)

In [8]:
df_wrong_wards = pd.DataFrame(wrong_wards)

In [9]:
df_wrong_wards = df_wrong_wards[(df_wrong_wards.province_key==df_wrong_wards.wrong_province_key) & (df_wrong_wards.district_key==df_wrong_wards.wrong_district_key)]

AttributeError: 'DataFrame' object has no attribute 'province_key'

In [ ]:
df_wrong_wards[df_wrong_wards.wrong_ward_key.isna()]

In [ ]:
df_wrong_wards

In [ ]:
DICT_double_check_wards = {}

for province_english in df_wrong_wards.province_english.unique():
    district_data = {}
    tmp_df_province = df_wrong_wards[df_wrong_wards.province_english ==province_english]
    for district_english in tmp_df_province.district_english.unique():
        tmp_df_district = tmp_df_province[tmp_df_province.district_english == district_english]
        ward_data = {}
        for wrong_ward_key in tmp_df_district.wrong_ward_key.unique():
            ward_data[wrong_ward_key] = tmp_df_district[tmp_df_district.wrong_ward_key == wrong_ward_key]['ward_key'].unique().tolist()
        
        ward_data = dict(sorted(ward_data.items(), key=lambda x: len(x[0]), reverse=True))
        district_data[district_english] = ward_data
    
    DICT_double_check_wards[province_english] = district_data

In [ ]:
DICT_double_check_wards